In [ ]:
import imaplib
import email
from email.header import decode_header

# Email Credentials
EMAIL = "your-email@example.com"
PASSWORD = "your-email-password"
IMAP_SERVER = "imap.gmail.com"  # For Gmail (Change for other providers)
FOLDER = "INBOX"

# Keywords indicating spam
SPAM_KEYWORDS = ["lottery", "winner", "prize", "free money", "claim now", "click here", "urgent", "congratulations", "you won"]

# Connect to the email server
def connect_email():
    try:
        mail = imaplib.IMAP4_SSL(IMAP_SERVER)
        mail.login(EMAIL, PASSWORD)
        print("[+] Logged in successfully")
        return mail
    except Exception as e:
        print("[-] Login failed:", e)
        return None

# Search for emails in the inbox
def fetch_emails(mail):
    mail.select(FOLDER)
    result, data = mail.search(None, "ALL")  # Fetch all emails
    if result != "OK":
        print("[-] Failed to fetch emails")
        return []
    return data[0].split()

# Check if an email is spam based on subject
def is_spam(mail, email_id):
    _, msg_data = mail.fetch(email_id, "(RFC822)")
    for response_part in msg_data:
        if isinstance(response_part, tuple):
            msg = email.message_from_bytes(response_part[1])
            subject, encoding = decode_header(msg["Subject"])[0]
            if isinstance(subject, bytes) and encoding:
                subject = subject.decode(encoding)
            
            # Check for spam keywords
            if subject:
                subject_lower = subject.lower()
                if any(spam_word in subject_lower for spam_word in SPAM_KEYWORDS):
                    print(f"[SPAM] Found spam email: {subject}")
                    return True
    return False

# Delete spam emails
def delete_spam_emails(mail):
    email_ids = fetch_emails(mail)
    spam_count = 0

    for email_id in email_ids:
        if is_spam(mail, email_id):
            mail.store(email_id, "+FLAGS", "\\Deleted")
            print(f"[-] Deleted spam email ID {email_id.decode()}")
            spam_count += 1
    
    # Expunge (permanently delete) marked emails
    mail.expunge()
    print(f"[+] Total spam emails deleted: {spam_count}")

# Main execution
if __name__ == "__main__":
    mail = connect_email()
    if mail:
        delete_spam_emails(mail)
        mail.logout()
